# The DICS beamformer

[Dynamic Imaging of Coherent Sources (DICS)](https://www.pnas.org/content/98/2/694) is a beamformer that localizes oscillatory activity at a specific frequency (range). It is closely related to the LCMV beamformer, but operates in the time-frequency domain.

We start by downloading the "somato" dataset that ships with MNE-Python. This dataset contains some nice oscillatory signals for us to analyze. The median nerve of the participant was stimulated at the left wrist, so we expect to see some event-related synchronization (ERS) in the beta band (12-30 Hz.)

In [ ]:
%matplotlib inline
import numpy as np
import mne
data_path = mne.datasets.somato.data_path(verbose=True)

## Sensor space analysis

Reading the raw data should look familiar. We save some memory by restricting our analysis to gradiometers only. Unfortunately, the DICS implementation in MNE-Python doesn't support mixing channel types yet.

In [ ]:
raw = mne.io.read_raw_fif(data_path + '/MEG/somato/sef_raw_sss.fif', preload=True)
raw = raw.pick_types(meg='grad', stim=True)

Constructing epochs. There is only one event (id=1) in the dataset. We use a generous time window surrounding the event.

In [ ]:
events = mne.find_events(raw)
epochs = mne.Epochs(raw, events, event_id=1, tmin=-1, tmax=3, preload=True)

Let's start our analysis by visualizing the time-frequency spectrum, i.e. the frequency content of the signal over time. MNE-Python supports three ways to compute this (short-time fourier transform, multitapers, and Morlet wavelets). In the demo, I'm going to use wavelets.

When using wavelets, you have to specify the exact frequencies you are interested in analysing. Since we're interested in the beta band, we define our frequencies from 12 to 30 Hz, spaced out using a logarithmic scale.

In [ ]:
# Frequencies of interest
freqs = np.logspace(np.log10(12), np.log10(30), 9)
print(freqs)

Let's compute the time-frequency decomposition of the signal!

In [ ]:
power, itc = mne.time_frequency.tfr_morlet(epochs, freqs=freqs, n_cycles=5, decim=10, n_jobs=2, verbose=True)

The [`plot_topo`](https://martinos.org/mne/stable/generated/mne.time_frequency.AverageTFR.html#mne.time_frequency.AverageTFR.plot_topo) method of the resulting [`AverageTFR`](https://martinos.org/mne/stable/generated/mne.time_frequency.AverageTFR.html#mne.time_frequency.AverageTFR) object provides a nice interactive visualization for us to explore.

We are usually interested in the relative *change in power* between some baseline period and what follows. For example, ERS is defined as an increase in power, relative to the baseline. Hence, we specify the baseline period in the plotting function and set it to `'logratio'` mode (plotting the log of the ratio of the post-baseline power versus the baseline).

In [ ]:
power.plot_topo(baseline=(-1, 0), mode='logratio', title='Average power')

## Source level analysis

We use the DICS beamformer to localize the source of the increase in beta activity. Like the first step for the LCMV beamformer was to compute a covariance matrix, the first step for DICS is to compute the cross-spectral density (CSD) matrix. It is the time-frequency equivalent of the covariance matrix.

As with the time-frequency decomposition earlier, we will use Morlet wavelets to compute the CSD.

In [ ]:
csd = mne.time_frequency.csd_morlet(epochs, freqs, tmin=-1, tmax=1.5, decim=10, n_jobs=2)

The CSD is computed for all the frequencies we defined earlier:

In [ ]:
csd.plot()

We can average across frequencies to obtain the CSD for a frequency *band*:

In [ ]:
csd.mean(12, 30).plot()

Loading the forward model for this dataset:

In [ ]:
fwd = mne.read_forward_solution(data_path + '/MEG/somato/somato-meg-oct-6-fwd.fif')

We are ready to compute the DICS spatial filters. We compute the filters using the forward model and the CSD.

In [ ]:
dics = mne.beamformer.make_dics(epochs.info, fwd, csd.mean(), pick_ori='max-power')

Now, the API with separate functions for `make_dics` and `apply_dics` comes into play: We have created the DICS spatial filters using a CSD that was computed across the entire time range. However, we want to make a contrast between the power before and after the stimulus onset.

So, we apply the beamformer to two distinct CSD matrices that we're going to compute for different time ranges.

In [ ]:
# Compute CSDs for two time intervals
# baseline:
csd_baseline = mne.time_frequency.csd_morlet(epochs, freqs, tmin=-1, tmax=0, decim=10, n_jobs=2)
# after stimulation:
csd_ers = mne.time_frequency.csd_morlet(epochs, freqs, tmin=0.5, tmax=1.5, decim=10, n_jobs=2)

In [ ]:
# Apply the DICS filters to the CSD matrices to obtain source power estimates
baseline_source_power, freqs = mne.beamformer.apply_dics_csd(csd_baseline.mean(), dics) 
beta_source_power, freqs = mne.beamformer.apply_dics_csd(csd_ers.mean(), dics)

What we are interested in is the relative change in power:

In [ ]:
%matplotlib qt
relative_source_power = beta_source_power / baseline_source_power
relative_source_power.plot(hemi='both', subjects_dir=data_path + '/subjects', views='par')

Stimulating the medial nerve of the left arm generates an increase in beta activity on the motor cortex in the right hemisphere.

<img src="images/ers.png" width="400">